#  Import the Necessary Libraries 

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
import random
import os
from google.cloud import bigquery
from google.oauth2 import service_account
import warnings
warnings.filterwarnings('ignore')

# Load the Global Superstore Data

In [2]:
orders_df = pd.read_excel('Global Superstore.xls', sheet_name='Orders')
returns_df = pd.read_excel('Global Superstore.xls', sheet_name='Returns')

print(" Orders loaded:", orders_df.shape)
print(" Returns loaded:", returns_df.shape)



 Orders loaded: (51290, 24)
 Returns loaded: (1173, 3)


# Exploaring the Global superstore Data

# Orders Sheet

In [3]:
orders_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [4]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row ID          51290 non-null  int64         
 1   Order ID        51290 non-null  object        
 2   Order Date      51290 non-null  datetime64[ns]
 3   Ship Date       51290 non-null  datetime64[ns]
 4   Ship Mode       51290 non-null  object        
 5   Customer ID     51290 non-null  object        
 6   Customer Name   51290 non-null  object        
 7   Segment         51290 non-null  object        
 8   City            51290 non-null  object        
 9   State           51290 non-null  object        
 10  Country         51290 non-null  object        
 11  Postal Code     9994 non-null   float64       
 12  Market          51290 non-null  object        
 13  Region          51290 non-null  object        
 14  Product ID      51290 non-null  object        
 15  Ca

In [5]:
orders_df.keys()

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country',
       'Postal Code', 'Market', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority'],
      dtype='object')

In [6]:
numerical = ["Sales", "Quantity", "Discount", "Profit", "Shipping Cost"]

categorical = ["Ship Mode","Segment","City","State","Country","Market","Region","Category","Sub-Category","Product Name"]

In [7]:
orders_df[categorical].describe()

,Ship Mode,Segment,City,State,Country,Market,Region,Category,Sub-Category,Product Name
count,51290,51290,51290,51290,51290,51290,51290,51290,51290,51290
unique,4,3,3636,1094,147,7,13,3,17,3788
top,Standard Class,Consumer,New York City,California,United States,APAC,Central,Office Supplies,Binders,Staples
freq,30775,26518,915,2001,9994,11002,11117,31273,6152,227


In [8]:
orders_df[numerical].describe()

,Sales,Quantity,Discount,Profit,Shipping Cost
count,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000
mean,246.490581,3.476545,0.142908,28.610982,26.375818
std,487.565361,2.278766,0.212280,174.340972,57.296810
min,0.444000,1.000000,0.000000,-6599.978000,0.002000
25%,30.758625,2.000000,0.000000,0.000000,2.610000
50%,85.053000,3.000000,0.000000,9.240000,7.790000
75%,251.053200,5.000000,0.200000,36.810000,24.450000
max,22638.480000,14.000000,0.850000,8399.976000,933.570000


In [9]:
orders_df["Ship Mode"].value_counts()

Ship Mode
Standard Class    30775
Second Class      10309
First Class        7505
Same Day           2701
Name: count, dtype: int64

In [10]:
orders_df["Segment"].value_counts()

Segment
Consumer       26518
Corporate      15429
Home Office     9343
Name: count, dtype: int64

In [11]:
orders_df["City"].unique()

array(['New York City', 'Wollongong', 'Brisbane', ..., 'Abilene',
       'Felahiye', 'Victoria Falls'], dtype=object)

In [12]:
orders_df["City"].value_counts()

City
New York City     915
Los Angeles       747
Philadelphia      537
San Francisco     510
Santo Domingo     443
                 ... 
Vesoul              1
Várzea Grande       1
Luanshya            1
Kissimmee           1
Victoria Falls      1
Name: count, Length: 3636, dtype: int64

In [13]:
orders_df["State"].unique()

array(['New York', 'New South Wales', 'Queensland', ..., 'Manicaland',
       'Kabarole', 'Matabeleland North'], dtype=object)

In [14]:
orders_df["State"].value_counts()

State
California            2001
England               1499
New York              1128
Texas                  985
Ile-de-France          981
                      ... 
Lambayeque               1
Aveiro                   1
Lima                     1
Medea                    1
Matabeleland North       1
Name: count, Length: 1094, dtype: int64

In [15]:
orders_df["Country"].unique()

array(['United States', 'Australia', 'Germany', 'Senegal', 'New Zealand',
       'Afghanistan', 'Saudi Arabia', 'Brazil', 'China', 'France',
       'Italy', 'Tanzania', 'Poland', 'United Kingdom', 'Mexico',
       'El Salvador', 'Taiwan', 'India', 'Dominican Republic',
       'Democratic Republic of the Congo', 'Indonesia', 'Uruguay', 'Iran',
       'Mozambique', 'Bangladesh', 'Spain', 'Ukraine', 'Nicaragua',
       'Morocco', 'Canada', 'Philippines', 'Austria', 'Colombia',
       'Netherlands', 'Malaysia', 'Ecuador', 'Thailand', 'Somalia',
       'Guatemala', 'Belarus', 'Cambodia', 'South Africa', 'Japan',
       'Russia', 'Egypt', 'Azerbaijan', 'Lithuania', 'Argentina',
       'Lesotho', 'Vietnam', 'Cuba', 'Romania', 'Turkey', 'Cameroon',
       'Hungary', 'Singapore', 'Angola', 'Belgium', 'Pakistan', 'Finland',
       'Ghana', 'Zambia', 'Iraq', 'Liberia', 'Georgia', 'Switzerland',
       'Albania', 'Chad', 'Montenegro', 'Namibia', 'Portugal',
       'Madagascar', 'Sweden', 'Myanmar 

In [16]:
orders_df["Market"].unique()

array(['US', 'APAC', 'EU', 'Africa', 'EMEA', 'LATAM', 'Canada'],
      dtype=object)

In [17]:
orders_df["Market"].value_counts()

Market
APAC      11002
LATAM     10294
EU        10000
US         9994
EMEA       5029
Africa     4587
Canada      384
Name: count, dtype: int64

In [18]:
orders_df["Region"].unique()

array(['East', 'Oceania', 'Central', 'Africa', 'West', 'South',
       'Central Asia', 'EMEA', 'North Asia', 'North', 'Caribbean',
       'Southeast Asia', 'Canada'], dtype=object)

In [19]:
orders_df["Region"].value_counts()

Region
Central           11117
South              6645
EMEA               5029
North              4785
Africa             4587
Oceania            3487
West               3203
Southeast Asia     3129
East               2848
North Asia         2338
Central Asia       2048
Caribbean          1690
Canada              384
Name: count, dtype: int64

In [20]:
orders_df["Category"].unique()

array(['Technology', 'Furniture', 'Office Supplies'], dtype=object)

In [21]:
orders_df["Category"].value_counts()

Category
Office Supplies    31273
Technology         10141
Furniture           9876
Name: count, dtype: int64

In [22]:
orders_df["Sub-Category"].unique()

array(['Accessories', 'Chairs', 'Phones', 'Copiers', 'Tables', 'Binders',
       'Supplies', 'Appliances', 'Machines', 'Bookcases', 'Storage',
       'Furnishings', 'Art', 'Paper', 'Envelopes', 'Fasteners', 'Labels'],
      dtype=object)

In [23]:
orders_df["Sub-Category"].value_counts()

Sub-Category
Binders        6152
Storage        5059
Art            4883
Paper          3538
Chairs         3434
Phones         3357
Furnishings    3170
Accessories    3075
Labels         2606
Envelopes      2435
Supplies       2425
Fasteners      2420
Bookcases      2411
Copiers        2223
Appliances     1755
Machines       1486
Tables          861
Name: count, dtype: int64

# Returns sheet

In [24]:
returns_df.head()

,Returned,Order ID,Market
0,Yes,MX-2013-168137,LATAM
1,Yes,US-2011-165316,LATAM
2,Yes,ES-2013-1525878,EU
3,Yes,CA-2013-118311,United States
4,Yes,ES-2011-1276768,EU


In [25]:
returns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Returned  1173 non-null   object
 1   Order ID  1173 non-null   object
 2   Market    1173 non-null   object
dtypes: object(3)
memory usage: 27.6+ KB


In [26]:
returns_df["Returned"].value_counts()

Returned
Yes    1173
Name: count, dtype: int64

In [27]:
returns_df["Market"].value_counts()

Market
LATAM            297
United States    296
APAC             296
EU               284
Name: count, dtype: int64

# Extracting the Products Data from Fakestore API

In [28]:
def get_products():
    url = 'https://fakestoreapi.com/products'
    response = requests.get(url)
    return response.json()

In [29]:
products = get_products()
products_df = pd.DataFrame(products)
products_df.head()

,id,title,price,description,category,image,rating
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,"{'rate': 3.9, 'count': 120}"
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,"{'rate': 4.1, 'count': 259}"
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,"{'rate': 4.7, 'count': 500}"
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,"{'rate': 2.1, 'count': 430}"
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,"{'rate': 4.6, 'count': 400}"


In [30]:
# Changing the columns name 

products_df = products_df.rename(columns={'id': 'product_id', 'title': 'product_name'})

products_df.head()


,product_id,product_name,price,description,category,image,rating
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,"{'rate': 3.9, 'count': 120}"
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,"{'rate': 4.1, 'count': 259}"
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,"{'rate': 4.7, 'count': 500}"
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,"{'rate': 2.1, 'count': 430}"
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,"{'rate': 4.6, 'count': 400}"


# Exploaring the Products Data

In [31]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    20 non-null     int64  
 1   product_name  20 non-null     object 
 2   price         20 non-null     float64
 3   description   20 non-null     object 
 4   category      20 non-null     object 
 5   image         20 non-null     object 
 6   rating        20 non-null     object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.2+ KB


In [32]:
products_df["product_name"].unique()

array(['Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops',
       'Mens Casual Premium Slim Fit T-Shirts ', 'Mens Cotton Jacket',
       'Mens Casual Slim Fit',
       "John Hardy Women's Legends Naga Gold & Silver Dragon Station Chain Bracelet",
       'Solid Gold Petite Micropave ', 'White Gold Plated Princess',
       'Pierced Owl Rose Gold Plated Stainless Steel Double',
       'WD 2TB Elements Portable External Hard Drive - USB 3.0 ',
       'SanDisk SSD PLUS 1TB Internal SSD - SATA III 6 Gb/s',
       'Silicon Power 256GB SSD 3D NAND A55 SLC Cache Performance Boost SATA III 2.5',
       'WD 4TB Gaming Drive Works with Playstation 4 Portable External Hard Drive',
       'Acer SB220Q bi 21.5 inches Full HD (1920 x 1080) IPS Ultra-Thin',
       'Samsung 49-Inch CHG90 144Hz Curved Gaming Monitor (LC49HG90DMNXZA) – Super Ultrawide Screen QLED ',
       "BIYLACLESEN Women's 3-in-1 Snowboard Jacket Winter Coats",
       "Lock and Love Women's Removable Hooded Faux Leather Moto Bike

In [33]:
products_df["category"].unique()

array(["men's clothing", 'jewelery', 'electronics', "women's clothing"],
      dtype=object)

In [34]:
products_df["category"].value_counts()

category
electronics         6
women's clothing    6
men's clothing      4
jewelery            4
Name: count, dtype: int64

In [35]:
products_df["rating"]

0     {'rate': 3.9, 'count': 120}
1     {'rate': 4.1, 'count': 259}
2     {'rate': 4.7, 'count': 500}
3     {'rate': 2.1, 'count': 430}
4     {'rate': 4.6, 'count': 400}
5      {'rate': 3.9, 'count': 70}
6       {'rate': 3, 'count': 400}
7     {'rate': 1.9, 'count': 100}
8     {'rate': 3.3, 'count': 203}
9     {'rate': 2.9, 'count': 470}
10    {'rate': 4.8, 'count': 319}
11    {'rate': 4.8, 'count': 400}
12    {'rate': 2.9, 'count': 250}
13    {'rate': 2.2, 'count': 140}
14    {'rate': 2.6, 'count': 235}
15    {'rate': 2.9, 'count': 340}
16    {'rate': 3.8, 'count': 679}
17    {'rate': 4.7, 'count': 130}
18    {'rate': 4.5, 'count': 146}
19    {'rate': 3.6, 'count': 145}
Name: rating, dtype: object

In [36]:
products_df["price"].describe()

count     20.000000
mean     162.046000
std      272.220532
min        7.950000
25%       15.240000
50%       56.490000
75%      110.962500
max      999.990000
Name: price, dtype: float64

# Simulate Inventory Levels

In [37]:

def simulate_inventory(products, days=30):
    inventory_history = []
    start_date = datetime.datetime.now() - datetime.timedelta(days=days)

    for product in products:
        
        # Initial inventory level
        
        base_inventory = random.randint(20, 200)
        daily_demand = random.randint(1, 10)

        for day in range(days):
            current_date = start_date + datetime.timedelta(days=day)
            
            # Simulate inventory reduction
            
            daily_sales = min(random.randint(0, daily_demand), base_inventory)
            base_inventory -= daily_sales
            
            
            # Simulate restocking every 7 days

            if day % 7 == 0:
                restock_amount = random.randint(30, 100)
                base_inventory += restock_amount

            inventory_history.append({
                'product_id': product['id'],
                'product_name': product['title'],
                'date': current_date.strftime('%Y-%m-%d'),
                'inventory_level': base_inventory,
                'category': product['category'],
                'price': product['price'],
                'restock': 1 if day % 7 == 0 else 0
            })

    return pd.DataFrame(inventory_history)


In [38]:
inventory_df = simulate_inventory(products)

inventory_df.head()

,product_id,product_name,date,inventory_level,category,price,restock
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",2025-07-04,255,men's clothing,109.95,1
1,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",2025-07-05,253,men's clothing,109.95,0
2,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",2025-07-06,252,men's clothing,109.95,0
3,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",2025-07-07,250,men's clothing,109.95,0
4,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",2025-07-08,250,men's clothing,109.95,0


# Exploaring the Inventory Data

In [39]:
inventory_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   product_id       600 non-null    int64  
 1   product_name     600 non-null    object 
 2   date             600 non-null    object 
 3   inventory_level  600 non-null    int64  
 4   category         600 non-null    object 
 5   price            600 non-null    float64
 6   restock          600 non-null    int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 32.9+ KB


In [40]:
inventory_df["product_name"].value_counts()

product_name
Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops                                                30
Mens Casual Premium Slim Fit T-Shirts                                                                30
Opna Women's Short Sleeve Moisture                                                                   30
MBJ Women's Solid Short Sleeve Boat Neck V                                                           30
Rain Jacket Women Windbreaker Striped Climbing Raincoats                                             30
Lock and Love Women's Removable Hooded Faux Leather Moto Biker Jacket                                30
BIYLACLESEN Women's 3-in-1 Snowboard Jacket Winter Coats                                             30
Samsung 49-Inch CHG90 144Hz Curved Gaming Monitor (LC49HG90DMNXZA) – Super Ultrawide Screen QLED     30
Acer SB220Q bi 21.5 inches Full HD (1920 x 1080) IPS Ultra-Thin                                      30
WD 4TB Gaming Drive Works with Playstation 4 Portab

In [41]:
inventory_df["inventory_level"].describe()

count    600.000000
mean     243.741667
std       90.380137
min       30.000000
25%      179.000000
50%      242.000000
75%      302.250000
max      468.000000
Name: inventory_level, dtype: float64

In [42]:
inventory_df["category"].value_counts()

category
electronics         180
women's clothing    180
men's clothing      120
jewelery            120
Name: count, dtype: int64

In [43]:
inventory_df["price"].describe()

count    600.000000
mean     162.046000
std      265.549137
min        7.950000
25%       15.240000
50%       56.490000
75%      110.962500
max      999.990000
Name: price, dtype: float64

In [44]:
inventory_df["restock"].value_counts()

restock
0    500
1    100
Name: count, dtype: int64

# Data cleaning and Transformation

In [45]:
# Section 5: Clean & Transform Global Superstore Orders

orders_df['Order Date'] = pd.to_datetime(orders_df['Order Date'], errors='coerce')
orders_df['Ship Date'] = pd.to_datetime(orders_df['Ship Date'], errors='coerce')


# Drop rows with missing dates

orders_df.dropna(subset=['Order Date', 'Ship Date'], inplace=True)


# Calculate Lead Time

orders_df['Lead Time (days)'] = (orders_df['Ship Date'] - orders_df['Order Date']).dt.days


# Simulate On-Time Delivery (lead time ≤ 5 days)

orders_df['On-time Delivery'] = orders_df['Lead Time (days)'] <= 5


# Mark Returns

orders_df['Returned'] = orders_df['Order ID'].isin(returns_df['Order ID'])

orders_df[['Order ID', 'Order Date', 'Ship Date', 'Lead Time (days)', 'On-time Delivery', 'Returned']].head()


,Order ID,Order Date,Ship Date,Lead Time (days),On-time Delivery,Returned
0,CA-2012-124891,2012-07-31,2012-07-31,0,True,False
1,IN-2013-77878,2013-02-05,2013-02-07,2,True,True
2,IN-2013-71249,2013-10-17,2013-10-18,1,True,False
3,ES-2013-1579342,2013-01-28,2013-01-30,2,True,False
4,SG-2013-4320,2013-11-05,2013-11-06,1,True,False


# Metrics Calculation

In [46]:

# Fill Rate (Orders not returned)

fill_rate = orders_df[~orders_df['Returned']].shape[0] / orders_df.shape[0]


# On-Time Delivery Rate

on_time_delivery_rate = orders_df['On-time Delivery'].mean()


print(f" Fill Rate: {fill_rate:.2%}")
print(f" On-Time Delivery Rate: {on_time_delivery_rate:.2%}")




 Fill Rate: 94.05%
 On-Time Delivery Rate: 81.84%


In [47]:
# Inventory Summary: average per product

avg_inventory = inventory_df.groupby('product_name')['inventory_level'].mean().sort_values(ascending=False)

print(" Average Inventory (Top 5):")

print(avg_inventory.head())

 Average Inventory (Top 5):
product_name
Pierced Owl Rose Gold Plated Stainless Steel Double                             354.766667
Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops                           340.966667
Rain Jacket Women Windbreaker Striped Climbing Raincoats                        324.900000
Mens Casual Slim Fit                                                            311.766667
Silicon Power 256GB SSD 3D NAND A55 SLC Cache Performance Boost SATA III 2.5    302.066667
Name: inventory_level, dtype: float64


# Saved the clean Data 

In [48]:

orders_df.to_csv("clean_orders.csv", index=False)
inventory_df.to_csv("simulated_inventory.csv", index=False)
products_df.to_csv("products.csv", index=False)

print("Data saved locally as CSV files.")


Data saved locally as CSV files.


# Connecting Python to Bigquery 

In [49]:

key_path = "supply-chain-data-467804-0aad25463359.json"
project_id = "supply-chain-data-467804"      
dataset_id = "supply_chain"          

# Authenticate
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=project_id)

print("Connected to BigQuery as:", client.project)


Connected to BigQuery as: supply-chain-data-467804


In [50]:
# Loading the CSV

In [51]:

orders_df = pd.read_csv("clean_orders.csv")
inventory_df = pd.read_csv("simulated_inventory.csv")
products_df = pd.read_csv("products.csv")

In [52]:
orders_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Lead Time (days),On-time Delivery,Returned
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical,0,True,False
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical,2,True,True
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium,1,True,False
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium,2,True,False
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical,1,True,False


In [53]:
# Changing the columns name in a correct format to upload in Bigquery

In [54]:
def clean_column_names(df):
    df.columns = (
        df.columns.str.strip()
                  .str.lower()
                  .str.replace(r"[^\w]+", "_", regex=True)
                  .str.replace(r"_+", "_", regex=True)
                  .str.strip("_")
    )
    return df


In [55]:
# Clean column names

orders_df = clean_column_names(orders_df)
inventory_df = clean_column_names(inventory_df)
products_df = clean_column_names(products_df)

In [56]:
orders_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,lead_time_days,on_time_delivery,returned
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical,0,True,False
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical,2,True,True
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium,1,True,False
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium,2,True,False
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical,1,True,False


In [57]:
orders_df.keys()

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'city', 'state', 'country',
       'postal_code', 'market', 'region', 'product_id', 'category',
       'sub_category', 'product_name', 'sales', 'quantity', 'discount',
       'profit', 'shipping_cost', 'order_priority', 'lead_time_days',
       'on_time_delivery', 'returned'],
      dtype='object')

In [58]:
# Creating a function to upload the data in bigquery if the table is already there it will overwrite that

In [59]:
def upload_table(df, table_name, schema=None):
    table_ref = f"{project_id}.{dataset_id}.{table_name}"
    job = client.load_table_from_dataframe(
        df, table_ref, job_config=bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_TRUNCATE")
    )
    job.result()
    print(f" Uploaded: {table_name}")

# Create DIM table

In [60]:
## dim_product

products_df.rename(columns={
    "id": "product_id",
    "title": "product_name",
    "category": "inventory_category"
}, inplace=True)
dim_product =products_df.drop_duplicates(subset="product_id").copy()

In [61]:
## dim_customer (from orders)

dim_customer = orders_df[["customer_id", "customer_name", "segment", "region", "market"]].drop_duplicates().copy()
dim_customer.rename(columns={
    "Customer ID": "customer_id",
    "Customer Name": "customer_name",
    "Segment": "segment",
    "Region": "region",
    "Market": "market"
}, inplace=True)

In [62]:
## dim_time

min_date = min(pd.to_datetime(orders_df["order_date"].min()), pd.to_datetime(orders_df["ship_date"].min()), pd.to_datetime(inventory_df["date"].min()))
max_date = max(pd.to_datetime(orders_df["order_date"].max()), pd.to_datetime(orders_df["ship_date"].max()), pd.to_datetime(inventory_df["date"].max()))
dates = pd.date_range(min_date, max_date)
dim_time = pd.DataFrame({"date": dates})
dim_time["year"] = dim_time["date"].dt.year
dim_time["month"] = dim_time["date"].dt.month_name()
dim_time["quarter"] = dim_time["date"].dt.quarter
dim_time["day"] = dim_time["date"].dt.day
dim_time["weekday"] = dim_time["date"].dt.day_name()
dim_time["week"] = dim_time["date"].dt.isocalendar().week

# Create FACT tables

In [63]:
## fact_orders

fact_orders = orders_df.copy()
fact_orders.rename(columns={
    "Order ID": "order_id",
    "Order Date": "order_date",
    "Ship Date": "ship_date",
    "Customer ID": "customer_id",
    "Product ID": "product_id",
    "Sales": "sales",
    "Quantity": "quantity",
    "Discount": "discount",
    "Profit": "profit",
    "Returned": "returned_flag",
    "On-time Delivery": "on_time_delivery",
    "Lead Time (days)": "lead_time_days"
}, inplace=True)


In [64]:
## fact_inventory

fact_inventory = inventory_df.copy()
fact_inventory.rename(columns={
    "id": "product_id"
}, inplace=True)


# Upload to BigQuery

In [65]:

upload_table(dim_product, "dim_product")
upload_table(dim_customer, "dim_customer")
upload_table(dim_time, "dim_time")
upload_table(fact_orders, "fact_orders")
upload_table(fact_inventory, "fact_inventory")

print("\n All tables uploaded to BigQuery")


 Uploaded: dim_product
 Uploaded: dim_customer
 Uploaded: dim_time
 Uploaded: fact_orders
 Uploaded: fact_inventory

 All tables uploaded to BigQuery


In [67]:
orders_df

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,lead_time_days,on_time_delivery,returned
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.570,Critical,0,True,False
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.630,Critical,2,True,True
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.490,Medium,1,True,False
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.160,Medium,2,True,False
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.040,Critical,1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,29002,IN-2014-62366,2014-06-19,2014-06-19,Same Day,KE-16420,Katrina Edelman,Corporate,Kure,Hiroshima,...,"Advantus Thumb Tacks, 12 Pack",65.100,5,0.0,4.5000,0.010,Medium,0,True,True
51286,35398,US-2014-102288,2014-06-20,2014-06-24,Standard Class,ZC-21910,Zuschuss Carroll,Consumer,Houston,Texas,...,Hoover Replacement Belt for Commercial Guardsm...,0.444,1,0.8,-1.1100,0.010,Medium,4,True,False
51287,40470,US-2013-155768,2013-12-02,2013-12-02,Same Day,LB-16795,Laurel Beltran,Home Office,Oxnard,California,...,"#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes",22.920,3,0.0,11.2308,0.010,High,0,True,False
51288,9596,MX-2012-140767,2012-02-18,2012-02-22,Standard Class,RB-19795,Ross Baird,Home Office,Valinhos,São Paulo,...,"Acco Index Tab, Economy",13.440,2,0.0,2.4000,0.003,Medium,4,True,False
